In [1]:
import numpy as np
import pandas as pd 
import sqlite3


In [2]:
con = sqlite3.connect('sharadar.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('SEP',), ('SF1',), ('TICKERS',), ('INDICATORS',), ('ACTIONS',), ('SF2',), ('SP500',), ('SF3A',), ('SF3B',), ('EVENTS',), ('DAILY',), ('SF3',), ('SFP',)]


In [24]:
%%time
sp500_raw = pd.read_sql_query("SELECT * FROM SP500", con)
sp500_raw['date'] = pd.to_datetime(sp500_raw['date'])
sp500 = sp500_raw.set_index(['date','ticker']).sort_index().drop(['action','contraticker','contraname','note'], axis=1)
sp500.to_csv('data/sp500_constituents.csv')
sp500

Wall time: 379 ms


name
date       ticker                                           
1957-03-04 AAMRQ                                    Amr Corp
           ABT                           Abbott Laboratories
           ACKHQ                      Armstrong Holdings Inc
           AD1                         Amsted Industries Inc
           AN1                                    Amoco Corp
...                                                      ...
2021-09-10 YUM                                Yum Brands Inc
           ZBH                    Zimmer Biomet Holdings Inc
           ZBRA                      Zebra Technologies Corp
           ZION    Zions Bancorporation National Association
           ZTS                                    Zoetis Inc

[49430 rows x 1 columns]

In [25]:
sec_mstr = sp500_raw[['ticker','name']].drop_duplicates(subset=['ticker']).reset_index().drop('index',axis=1)
sec_mstr.to_csv('data/sec_master.csv')
sec_list = sec_mstr.ticker.tolist()
sec_mstr

,ticker,name
0,A,Agilent Technologies Inc
1,AAL,American Airlines Group Inc
2,AAP,Advance Auto Parts Inc
3,AAPL,Apple Inc
4,ABBV,AbbVie Inc
...,...,...
1119,HW1,Howmet Corp
1120,FALB1,Falstaff Brewing Corp
1121,ESB1,Esb Inc
1122,CGW2,Chicago Great Western Rail Co


In [26]:
%%time
query = f"SELECT * FROM SF1 WHERE ticker in ({','.join(['?']*len(sec_list))})"
cursor.execute(query, sec_list)
sf1_raw = pd.DataFrame(cursor.fetchall())
sf1_raw.columns = list(map(lambda x: x[0], cursor.description))
sf1_raw['datekey'] = pd.to_datetime(sf1_raw['calendardate'])
sf1 = sf1_raw.drop(['dimension','reportperiod','lastupdated','datekey'], axis=1)
sf1 = sf1.set_index(['calendardate','ticker']).sort_index()
sf1.to_csv('data/sf1.csv')
sf1

accoci       assets    assetsavg     assetsc  \
calendardate ticker                                                     
1991-06-30   DEC                                                        
1991-09-30   DIS                                                        
             HPQ                                                        
1991-12-31   AXP                                                        
             AXP                                                        
...                         ...          ...          ...         ...   
2021-12-31   WDC     -197000000  26132000000  25623750000  9757000000   
             WOR       45387000   3373245000   3167389000  1967992000   
             WOR       45387000   3373245000   3167389000  1967992000   
             XLNX      -2113000   5519201000   5572215000  3746262000   
             XLNX      -2113000   5519201000   5572215000  3746262000   

                        assetsnc assetturnover    bvps        capex  \
calendardate ticker                                                   
1991-06-30   DEC                                         -737548000   
1991-09-30   DIS                                          924600000   
             HPQ                                         -699000000   
1991-12-31   AXP                                         -513000000   
             AXP                                         -513000000   
...                          ...           ...     ...          ...   
2021-12-31   WDC     16375000000          0.66  35.151  -1003000000   
             WOR      1405253000         1.001  26.531    -82178000   
             WOR      1405253000         1.001  26.531    -82178000   
             XLNX     1772939000         0.565  11.819    -49665000   
             XLNX     1772939000         0.565  11.819    -49665000   

                        cashneq  cashnequsd  ...  sharesbas    shareswa  \
calendardate ticker                          ...                          
1991-06-30   DEC                             ...              121558000   
1991-09-30   DIS                             ...             1598100000   
             HPQ                             ...                          
1991-12-31   AXP                             ...                          
             AXP                             ...                          
...                         ...         ...  ...        ...         ...   
2021-12-31   WDC     3370000000  3370000000  ...  306452886   305000000   
             WOR      640311000   640311000  ...   51741899    52701000   
             WOR      640311000   640311000  ...   52990921    52701000   
             XLNX    1438528000  1438528000  ...  245840000   244257000   
             XLNX    1438528000  1438528000  ...  245277000   244257000   

                    shareswadil      sps    tangibles taxassets     taxexp  \
calendardate ticker                                                          
1991-06-30   DEC      121558000  114.439                          97707000   
1991-09-30   DIS     1598100000    3.825                         375300000   
             HPQ                                                 372000000   
1991-12-31   AXP                                                  15000000   
             AXP                                                  15000000   
...                         ...      ...          ...       ...        ...   
2021-12-31   WDC      309000000   55.482  15624000000         0  106000000   
             WOR       53917000   60.178   2781802000   1958000  176267000   
             WOR       53917000   60.178   2781802000   1958000  176267000   
             XLNX     247229000   12.886   4726912000         0   83170000   
             XLNX     247229000   12.886   4726912000         0   83170000   

                    taxliabilities   tbvps workingcapital  
calendardate ticker                                        
1991-06-30   DEC                        

In [28]:
%%time
query = f"SELECT * FROM SEP WHERE ticker in ({','.join(['?']*len(sec_list))})"
cursor.execute(query, sec_list)
sep_raw = pd.DataFrame(cursor.fetchall())
sep_raw.columns = list(map(lambda x: x[0], cursor.description))
sep_raw['date'] = pd.to_datetime(sep_raw['date'])
sep = sep_raw.drop(['lastupdated'], axis=1)
sep = sep.set_index(['date','ticker']).sort_index()
sep.to_csv('data/sep.csv')
sep

Wall time: 1min 19s


open     high      low    close        volume  closeadj  \
date       ticker                                                               
1997-12-31 AAMRQ    62.969   64.938   62.750   64.250  3.793387e+06    32.423   
           AAPL      0.117    0.122    0.116    0.117  4.063584e+08     0.101   
           ABC       7.125    7.414    7.125    7.281  5.168000e+05     5.684   
           ABI1     17.437   17.828   17.047   17.766  6.391998e+05    16.580   
           ABKFQ    30.000   30.750   29.917   30.667  2.562000e+05    30.667   
...                    ...      ...      ...      ...           ...       ...   
2021-09-13 YUM     130.200  130.260  127.760  129.050  1.659227e+06   129.050   
           ZBH     140.890  142.520  140.500  141.000  1.504697e+06   141.000   
           ZBRA    587.690  587.690  575.000  579.900  1.415450e+05   579.900   
           ZION     58.120   58.770   57.560   58.310  1.260154e+06    58.310   
           ZTS     208.550  208.990  202.210  205.000  1.407853e+06   205.000   

                   closeunadj  
date       ticker              
1997-12-31 AAMRQ      128.500  
           AAPL        13.130  
           ABC         58.250  
           ABI1        71.062  
           ABKFQ       46.000  
...                       ...  
2021-09-13 YUM        129.050  
           ZBH        141.000  
           ZBRA       579.900  
           ZION        58.310  
           ZTS        205.000  

[4482822 rows x 7 columns]

In [29]:
%%time
query = f"SELECT * FROM DAILY WHERE ticker in ({','.join(['?']*len(sec_list))})"
cursor.execute(query, sec_list)
financials_raw = pd.DataFrame(cursor.fetchall())
financials_raw.columns = list(map(lambda x: x[0], cursor.description))
financials_raw['date'] = pd.to_datetime(financials_raw['date'])
financials = financials_raw.drop(['lastupdated'], axis=1)
financials = financials.set_index(['date','ticker']).sort_index()
financials.to_csv('data/financials.csv')
financials

Wall time: 1min 11s


ev evebit evebitda marketcap    pb    pe    ps
date       ticker                                                      
1998-12-01 AAMRQ    16619.5    8.1      5.1   12376.5   1.9  12.6   0.7
           AAPL      4349.3   -4.2     -4.7    4599.3   3.1  -4.4   0.6
           ABC       1962.3   15.0     13.4    1537.3  22.3  30.3   0.2
           ABI1      4579.8   46.4     29.9    4563.2   7.9  84.2   2.9
           ABKFQ     4613.9   15.0     14.9    4199.6   2.0  18.8  11.0
...                     ...    ...      ...       ...   ...   ...   ...
2021-09-14 YUM      48241.4   23.3     21.6   38135.4  -4.8  28.6   6.1
           ZBH      37651.7   35.7     17.8   30841.6   2.5  33.7   3.9
           ZBRA     31593.3   35.5     29.9   30802.3  11.9  40.4   6.0
           ZION       653.3    0.5      0.4    9215.3   1.1   8.4   2.9
           ZTS     100156.3   38.9     33.2   96471.3  22.2  50.5  13.0

[4222897 rows x 7 columns]